<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%%capture
!pip install eikon
!pip install wandb

In [ ]:
import eikon as ek
import wandb
import pandas as pd
import numpy as np
import time
import os
import glob
from tqdm.auto import tqdm


In [ ]:
#"api_key_lk"
ek.set_app_key("example")

## Screen companies

In [ ]:
mic_exchanges = pd.read_csv("mic_codes.csv").set_index("MIC") #Can be used to look up specific stock exchanges codes :) (Y) 

In [ ]:
oil_osebx_screen = 'SCREEN(U(IN(Equity(active,public,primary))), TR.CompanyMarketCap>=500000, IN(TR.ExchangeMarketIdCode,"XOSL"), IN(TR.TRBCBusinessSectorCode,"5010","5020","5030"), CURN=USD)'
fields_oil_osebx_screen = ["TR.CommonName"]#["TR.CommonName","TR.CompanyMarketCap","TR.ExchangeName","TR.TRBCBusinessSector","TR.TotalReturn3Mo"]

osbx_companies, e = ek.get_data(oil_osebx_screen, fields_oil_osebx_screen)
osbx_companies = osbx_companies.set_index("Instrument")

In [ ]:
oil_global_screen = 'SCREEN(U(IN(Equity(active,public,primary))), TR.CompanyMarketCap>=500000, IN(TR.TRBCBusinessSectorCode,"5010","5020","5030"), CURN=USD)'
fields_oil_global_screen = ["TR.CommonName"]

global_oil, e = ek.get_data(oil_global_screen, fields_oil_global_screen)
global_oil = global_oil.set_index("Instrument")

Now we have dataframe of all noted oil companies (with mcap > USD 5m) in eikon refinitives entire database

# Collect data from eikon refinitiv

In [ ]:
######## INPUTS ########
lst_of_tickers = global_oil.index.to_list()

#Eikon parameters
start_date = '2000-01-01'
end_date = '2022-04-21'
ek_params = {'SDate': start_date, 'EDate': end_date,'Frq': 'FQ', "Curn":"USD"}

#Max http company request at once
search_limit = 10_000

#What data to get
get_stock_data = False
get_meta_data = True
get_fundamental_data = False 
get_broker_data = False

toggle_dict = {'stock_data':get_stock_data, 'meta_data':get_meta_data,
               'fundamental_data':get_fundamental_data, 'broker_data':get_broker_data}



params = ek_params | toggle_dict | {'limit': search_limit} 
########################

## We define functions to find stock, meta, fundamental and broker estimates data 

In [ ]:
def _sub_lists(data, size_m):
    return [data[x:x+size_m] for x in range(0, len(data), size_m)]

#Function to counteract http timeout
def _divide_pull_request(lst_of_tickers, fields, params, suffix):

    p = {key: val for key, val in params.items() if key in ek_params}    
    if len(lst_of_tickers) > params['limit']:
        dfs = []
        for sub_ticker_lst in tqdm(_sub_lists(lst_of_tickers, params['limit']), suffix):
            df_sub, err = ek.get_data(lst_of_tickers, fields, p)
            print(df_sub)
            dfs.append(df_sub)
        df = pd.concat(dfs, axis=0)
    else: 
        df, err = ek.get_data(lst_of_tickers, fields, p)
    return df

In [ ]:
def stock_data(lst_of_tickers, params):
    
    params_new = params.copy()
    params_new['Frq'] = 'D'
    
    fields = ['TR.CompanyMarketCap.Date','TR.CompanyMarketCap', 'TR.PriceClose',
              'TR.CompanyMarketCap.Currency'] #TR.F.ComShrOutsTot
    
    stock_df = _divide_pull_request(lst_of_tickers, fields=fields, params=params_new, suffix=' Getting time series')
    
    return stock_df

In [ ]:
#Meta data collector
def meta_data(lst_of_tickers):
    geography = ['TR.ExchangeMarketIdCode', 'TR.HeadquartersRegionAlt', 'TR.HeadquartersCountry', 'TR.HQStateProvince']
    sectors = ['TR.TRBCEconomicSector', 'TR.TRBCBusinessSector', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustry', 'TR.TRBCActivity']
    founded = ['TR.OrgFoundedYear']

    meta_data = geography + founded + sectors  
    meta_df, _ = ek.get_data(lst_of_tickers, meta_data)
    meta_df = meta_df.set_index("Instrument")
    
    meta_df['Organization Founded Year'] = meta_df['Organization Founded Year'].replace(0, np.NaN) #<-- Eikon hilariously uses 0 instead of Na for missing year value
    
    
    return meta_df

In [ ]:
#Fundamental data collector

def fundamental_data(lst_of_tickers, params):
    #fields    
    profits = ['TR.TotalRevenue', 'TR.GrossProfit','TR.EBITDA','TR.EBIT', 'TR.F.NetIncAfterTax']#, 'TR.EV','MKT_CAP']
    balance = ['TR.F.TotAssets','TR.F.TotCurrAssets','TR.F.TotLiab','TR.F.TotCurrLiab','TR.F.LTDebtPctofTotAssets','TR.F.STDebtPctofTotAssets']#TR.F.TotLiab(Period=FY0)
    cash_flow = ['TR.F.LeveredFOCF']
    fundamental_data = profits + balance + cash_flow 
    
    other = []#['TR.InsiderBuyDepthComp'] <--- NA only, could be interesting to use....   
    reported_dates = ['TR.TotalRevenue.date','TR.TotalRevenue.periodenddate','TR.BSOriginalAnnouncementDate']
    
    fields = reported_dates + fundamental_data + other
    
    #collect data
    fundamental_df = _divide_pull_request(lst_of_tickers, fields, params, suffix=' Getting fundamentals')
    
    return fundamental_df

In [ ]:
def broker_estimates(lst_of_tickers, params):
    
    params_new = params.copy()
    params_new["Period"] = "FY1"    
    
    fields = ["TR.EPSMean","TR.EPSMean.periodenddate","TR.EBITMean",'TR.RevenueMean',
              "TR.ROAMean","TR.ROEMean","TR.FCFMean","TR.TotalAssets","TR.MeanPctChg(Period=FY1,WP=60d)"]
    
    estimates_df, err = ek.get_data(lst_of_tickers, fields, params)
    return estimates_df

In [ ]:
def get_data(lst_of_tickers, params):
    
    stock_df = None
    meta_df = None
    fundamental_df = None
    broker_df = None
    
    if params['stock_data']: 
        stock_df = stock_data(lst_of_tickers, params)

    if params['meta_data']:
        meta_df = meta_data(lst_of_tickers)

    if params['fundamental_data']:
        fundamental_df = fundamental_data(lst_of_tickers, params)

    if params['broker_data']:
        broker_df = broker_estiqates(lst_of_tickers, params)
    
    return stock_df, meta_df, fundamental_df, broker_df
        

In [ ]:
def save_data(file_name, save_per_n_http_request, lst_of_tickers, params):
    
    non_collected_tickers = []

    name_to_index = {}
    dfs = {}
    for i, possible_key in enumerate(["stock_data", "meta_data", "fundamental_data", "broker_data"]):
        if params[possible_key]:
            name_to_index[possible_key] = i
            dfs[possible_key] = []
    
    partioned_lst_of_tickers = _sub_lists(lst_of_tickers, params["limit"])        
    for i, sub_ticker_lst in enumerate(tqdm(partioned_lst_of_tickers, "saving loop")):    
        
        try:
            raw_data_dfs = get_data(sub_ticker_lst, params)
            
            for key in name_to_index:
           
                dfs[key] = dfs[key] + [raw_data_dfs[name_to_index[key]]]
                
                
                
            if not (i % save_per_n_http_request):
                for key in name_to_index:
                    df = pd.concat(dfs[key], axis=0)
                    df = df.reset_index()
                    
                    df.to_feather(f"{file_name}_save={i}_type={key}.feather")
                    
                    
                    dfs[key] = []

        except ek.EikonError as err:
            for key in name_to_index:
                dfs[key] = []
                
            non_collected_tickers += sub_ticker_lst
        except Exception as e:
            print(e)
            for key in name_to_index:
                dfs[key] = []
                
            non_collected_tickers += sub_ticker_lst
            
    #Write crashes to file       
    with open(f"{file_name}.txt", "w") as f:
        f.write("\n".join(non_collected_tickers))
    
    #Save last data if there are rests
    for key in name_to_index:
            break
            if dfs[key] != []:
                df = pd.concat(dfs[key], axis=0)

                df = df.reset_index()

                #wtfffff
                df.to_feather(f"{file_name}_save={len(partioned_lst_of_tickers)}_type={key}.feather")

            

In [ ]:
save_toggle = False

file_name = "C:/Users/kjartkra/Untitled Folder/meta_data/global_oil"
if save_toggle: 
    save_data(file_name, 1, lst_of_tickers, params)

In [ ]:
def _time_interval(start_date, end_date):
    y0 = int(start_date.split("-")[0])
    yn = int(end_date.split("-")[0])
    in_between_dates = [f"{str(year)}-01-01" for year in range(y0+1,yn,7)]
    return [start_date] + in_between_dates  + [end_date]
    
        

def macro_data(lst_of_tickers, ek_get_timeseries_fields, params):
        start_and_ends = _time_interval(params["SDate"],params["EDate"])
        
        tickers_to_serie = {}
        for ticker in lst_of_tickers:
            tickers_to_serie[ticker] = []
            for i in range(len(start_and_ends)-1):
                try: 
                    time_series = ek.get_timeseries(ticker, fields=ek_get_timeseries_fields,
                                                    start_date=start_and_ends[i], end_date=start_and_ends[i+1], interval=params["interval"])
                                 
                except ek.EikonError as err:
                    if err.code ==-1:
                        time_series = ek.get_timeseries("BRT-", fields=ek_get_timeseries_fields, start_date=start_and_ends[i], end_date=start_and_ends[i+1],interval=params["interval"])
                        time_series[ek_get_timeseries_fields] = np.nan
                        
                    if err.code ==  2504:
                        print("backend error")
                        time.sleep(2)
                        time_series = ek.get_timeseries(ticker, fields=ek_get_timeseries_fields,
                                                    start_date=start_and_ends[i], end_date=start_and_ends[i+1], interval=params["interval"])
                        
                tickers_to_serie[ticker] = tickers_to_serie[ticker] + [time_series]
            
            tickers_to_serie[ticker] = pd.concat(tickers_to_serie[ticker], axis=0)
            
        return tickers_to_serie

In [ ]:
def dict_to_df(dictionary):
    dates = set()

    for key, frame in dictionary.items():
        dates |= set(frame.index.values)
    
    index = pd.Index(list(sorted(dates)))
    
    all_macro = pd.DataFrame(index=index)

    for key, frame in dictionary.items():
        frame = frame[~frame.index.duplicated(keep='first')]
        all_macro[key] = frame

    return all_macro

In [ ]:
def folder_to_df(folder_with_data):
    files = glob.glob(folder_with_data + '/*.feather')
    dfs = []
    for file in files:
        dfs.append(pd.read_feather(file).set_index("index"))

    df_big = pd.concat(dfs, axis=0).reset_index()
    df_big = df_big.drop("index", axis=1)
    return df_big

In [ ]:
def upload_artifact(run, dataframe_file_location, artifact_name):
    
    artifact = wandb.Artifact(artifact_name, type='dataset')

    # Add a file to the artifact's contents
    artifact.add_file(dataframe_file_location)

    # Save the artifact version to W&B and mark it as the output of this run
    run.log_artifact(artifact)

In [ ]:
collect_meta_data = False
if collect_meta_data:
    stock_df, meta_df, fundamental_df, broker_df = get_data(lst_of_tickers, params)

    meta_location = 'C:/Users/kjartkra/Untitled Folder/meta_oil.feather'
    meta_df.reset_index().to_feather(meta_location)

In [ ]:
collect_macro_data = False

if collect_macro_data:
    macro_oil_params = ek_params.copy()
    macro_oil_params["interval"] = "daily"
    macro_oil_series = ["BRT-", "CLc1", "WTCLc1", "LNG-AS", ".VIX",'EUR=', 'GBP=', "CNY=", ]
    macro_oil_fields = ["CLOSE"]

    macro_oil = macro_data(macro_oil_series, macro_oil_fields , macro_oil_params)

In [ ]:
fundamentals_df = folder_to_df('C:/Users/kjartkra/Untitled Folder/fundamental_data')
fundamentals_location = 'C:/Users/kjartkra/Untitled Folder/fundamentals_oil.feather'

fundamentals_df.to_feather(fundamentals_location)

In [ ]:
macro_df = dict_to_df(macro_oil).reset_index()
oil_company_df = folder_to_df("C:/Users/kjartkra/Untitled Folder/stock_data/)

macro_location = 'C:/Users/kjartkra/Untitled Folder/macro_oil.feather'
company_location = 'C:/Users/kjartkra/Untitled Folder/companies_oil.feather'

macro_df.to_feather(macro_location)
oil_company_df.to_feather(company_location)


In [ ]:
upload_stocks = False
upload_meta = True
upload_fundamentals = False
upload_macro = False


if  upload_stocks or upload_meta or upload_fundamentals or upload_macro:
    with wandb.init(project="master-test") as run:
        if upload_stocks:
            upload_artifact(run, company_location, "oil-company-data")
        if upload_meta: 
            upload_artifact(run, meta_location, "oil-meta-data")
        if upload_fundamentals:
            upload_artifact(run, fundamentals_location, "oil-fundamental-data")
        if upload_macro:
            upload_artifact(run, macro_location, "oil-macro-data")
      

## Testing

In [ ]:
df_big

In [ ]:
wandb.init()

artifact = wandb.Artifact('mnist', type='dataset')
artifact.add_dir('mnist/')
wandb.log_artifact(artifact)

In [ ]:
time_series_df_2, meta_df, fundamental_df, broker_df = get_data(lst_of_tickers[:5], params)
time_series_df_2

In [ ]:
time_series_df_2["Number Of Stocks"].isna().sum()

In [ ]:
time_series_df["Common Shares - Outstanding - Total"].isna().sum()

In [ ]:
time_series_df

In [ ]:
pd.set_option('display.max_rows', 1000)
time_series_df.to_excel("stock_data_2.xlsx")

In [ ]:
#Conclusion, makes small difference in time to process at server
test_time = False

if test_time: 
    params_single = {'SDate': start_date, 'EDate': end_date,'Frq': 'FQ','Period': 'FQ0'}
    params_curn = {'SDate': start_date, 'EDate': end_date,'Frq': 'FQ','Period': 'FQ0', "Curn":"USD"}
    
    
    start_time = time.time()
    data,err = ek.get_data(osbx_companies.index[:3].to_list(), financials, params_single)
    print("--- simple: %s seconds ---" % (time.time() - start_time))

    start_time = time.time()
    data_usd,err = ek.get_data(osbx_companies.index[:3].to_list(), financials, params_curn)
    print("--- Curn: %s seconds ---" % (time.time() - start_time))

    start_time = time.time()
    data_all,err = ek.get_data(osbx_companies.index[:3].to_list(), financials, params)
    print("--- Scale & Curn: %s seconds ---" % (time.time() - start_time))